In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix/data


In [0]:
labels = pd.read_csv('sign_labels.csv')
labels_dict = labels.to_dict()['b']

In [0]:
train = pd.read_pickle('sign_train.p')
test = pd.read_pickle('sign_test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:  y_train = to_categorical(y_train)
if y_test.ndim == 1:  y_test = to_categorical(y_test)

In [6]:
y_train.shape, y_test.shape

((34799, 43), (4410, 43))

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_model(input_shape, num_classes):
  model = Sequential([
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                      MaxPool2D(),
                      Dropout(0.3),

                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(0.3),

                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                      MaxPool2D(),
                      Dropout(0.3),                      

                      Flatten(),

                      Dense(1024, activation='relu'),
                      Dropout(0.3),

                      Dense(1024, activation='relu'),
                      Dropout(0.3),
                      
                      Dense(num_classes, activation='softmax')                
  ])

  return model

In [0]:
model = get_model(input_shape, num_classes)

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensor_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensor_callback]
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(get_model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [17]:
train_and_predict( get_model(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.6315 - accuracy: 0.2608 - val_loss: 1.4438 - val_accuracy: 0.5281
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 1.1094 - accuracy: 0.6360 - val_loss: 0.2891 - val_accuracy: 0.9106
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.5009 - accuracy: 0.8403 - val_loss: 0.1044 - val_accuracy: 0.9701
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3404 - accuracy: 0.8962 - val_loss: 0.0714 - val_accuracy: 0.9799
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2346 - accuracy: 0.9290 - val_loss: 0.0572 - val_accuracy: 0.9841


0.962358276643991

In [0]:
!git config --global user.email "m.zajac1988@gmail.com"
!git config --global user.name "Mateusz"

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three


In [0]:
!git add day3.ipynb

In [0]:
!git commit -m 'First CNN Attempt on Road Signs'

[master 1f2b1db] First CNN Attempt on Road Signs
 1 file changed, 1 insertion(+)
 create mode 100644 matrix_three/day3.ipynb


In [0]:
!git push -u origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 2.48 MiB | 1.19 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/mattzajac/dw_matrix.git
   18f1340..1f2b1db  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
